In [4]:
!pip install -r requeriments.txt

In [1]:
import tenseal as ts
import os
import pickle
import numpy as np

In [80]:
# context = ts.context(
#     ts.SCHEME_TYPE.CKKS,   # Esquema CKKS para valores de ponto flutuante
#     8192,                  # poly_modulus_degree = 4096
#     [40, 21, 21, 40]       # Coefficient modulus bits sizes (adequado para CKKS de 4096)
# )

In [2]:
def context():
    """
    This function is used to create the context of the homomorphic encryption:
    it is used to create the keys and the parameters of the encryption scheme (CKKS).

    :return: the context of the homomorphic encryption
    """
    cont = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        # This means that the coefficient modulus will contain 4 primes of 60 bits, 40 bits, 40 bits, and 60 bits.
        coeff_mod_bit_sizes=[60, 40, 40, 60]
        # coeff_mod_bit_sizes=[60, 40, 40, 60]
    )

    cont.generate_galois_keys()  # You can create the Galois keys by calling generate_galois_keys
    # cont.generate_relin_keys()
    cont.global_scale = 2 ** 40  # global_scale: the scaling factor, here set to 2**40 (same that pow(2, 40))
    return cont

In [3]:
def read_query(file_path):
    """
    This function is used to read a pickle file.

    :param file_path: the path of the file to read
    :return: the query and the context
    """
    if os.path.exists(file_path):
        with open(file_path, 'rb') as file:
            """
            # pickle.load(f)  # load to read file object

            file_str = f.read()
            client_query1 = pickle.loads(file_str)  # loads to read str class
            """
            query_str = pickle.load(file)

        contexte = query_str["context"]  # ts.context_from(query["contexte"])
        del query_str["context"]
        return query_str, contexte

    else:
        print("The file doesn't exist")

In [4]:
def combo_keys(client_path="secret.pkl", server_path="server_key.pkl"):
    """
    To create the public/private keys combination
    args:
        client_path: path to save the secret key (str)
        server_path: path to save the server public key (str)
    """
    context_client = context()
    write_query(f'context/{client_path}', {"context": context_client.serialize(save_secret_key=True)})
    write_query(f'context/{server_path}', {"context": context_client.serialize()})

    _, context_client = read_query(f'context/{client_path}')
    _, context_server = read_query(f'context/{server_path}')

    context_client = ts.context_from(context_client)
    context_server = ts.context_from(context_server)
    print("Is the client context private?", ("Yes" if context_client.is_private() else "No"))
    print("Is the server context private?", ("Yes" if context_server.is_private() else "No"))

In [5]:
def write_query(file_path, client_query):
    """
    This function is used to write a pickle file.

    :param file_path: the path of the file to write

    :param client_query: the query to write
    """
    with open(file_path, 'wb') as file:  # 'ab' to add existing file
        encode_str = pickle.dumps(client_query)
        file.write(encode_str)

In [6]:
combo_keys()

Is the client context private? Yes
Is the server context private? No


In [7]:
with open('context/secret.pkl', 'rb') as file:
    secret = pickle.load(file)

context_client = ts.context_from(secret["context"])

In [8]:
with open('context/server_key.pkl', 'rb') as file:
    secret = pickle.load(file)

context_server = ts.context_from(secret["context"])

In [9]:
context_client.secret_key()

In [10]:
import tensorflow as tf

model = tf.keras.models.Sequential([
            tf.keras.layers.Input(shape=(10, 10, 1)),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(10, activation='relu'),
            tf.keras.layers.Dense(64,  activation='relu'),
            tf.keras.layers.Dense(32,  activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax'),

        ])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

2024-12-04 13:13:28.875862: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-04 13:13:29.071024: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-04 13:13:29.101495: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/allanmsouza/anaconda3/lib/:/home/allanmsouza/anaconda3/lib/:/home/allanmsouz

In [11]:
def flat_model(model):
    flat_model = []
    pesos      = model.get_weights()

    for p in pesos:
        flat_model.extend(p.flatten())

    return flat_model

flat_parameters = flat_model(model)

In [73]:
a = flat_parameters[:4096]
print(a[0], type(a[0]))

0.1984534 <class 'numpy.float32'>


In [74]:
he_tensor = ts.ckks_tensor(context_client, a) 
he_vector = ts.ckks_vector(context_client, a)

he_tensor *= 1
he_vector *= 1

In [75]:
import sys

print(sys.getsizeof(he_vector.serialize())/(1024 * 1024))
print(sys.getsizeof(he_tensor.serialize())/(1024 * 1024))


0.22448062896728516
919.3182287216187


In [76]:
decrypt_vec = he_vector.decrypt()
decrypt_ten = he_tensor.decrypt().raw

for _ in range(len(a)):
    print(f'Original: {a[_]} Vec: {decrypt_vec[_]} Tensor: {decrypt_ten[_]}')

Original: 0.1984533965587616 Vec: 0.19845342311424707 Tensor: 0.19845342393794593
Original: 0.15283963084220886 Vec: 0.15283965176999825 Tensor: 0.15283965088220425
Original: 0.16315296292304993 Vec: 0.1631529850608396 Tensor: 0.16315298491890146
Original: -0.1939784586429596 Vec: -0.1939784859952296 Tensor: -0.19397848500188972
Original: 0.11377805471420288 Vec: 0.11377806834254288 Tensor: 0.11377807169078923
Original: -0.03126727044582367 Vec: -0.03126727688289846 Tensor: -0.03126727500639201
Original: 0.15959864854812622 Vec: 0.1595986722352573 Tensor: 0.15959867083694151
Original: 0.06510218977928162 Vec: 0.06510219788123056 Tensor: 0.06510219878049361
Original: 0.10123801231384277 Vec: 0.10123802747816601 Tensor: 0.10123802642168811
Original: -0.11852104961872101 Vec: -0.11852106275783109 Tensor: -0.11852106636197798
Original: -0.09553851187229156 Vec: -0.09553852493025898 Tensor: -0.0955385252679353
Original: -0.04249550402164459 Vec: -0.04249551119849422 Tensor: -0.0424955088324

: 

In [58]:
he_layer = ts.ckks_vector(context_client, flat_parameters)

The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.


In [60]:
# a = ts.ckks_tensor(context_client,[1/10])

he_layer *= 2
print(he_layer.decrypt())

[0.7938142225536408, 0.6113590252034224, 0.6526123757710864, -0.7759144598047911, 0.4551125928505258, -0.12506916604449986, 0.6383951064235994, 0.2604089743126033, 0.4049523507141356, -0.4740845685529661, -0.3821543594018099, -0.1699821588211028, -0.7427592888097392, 0.5596224892317146, -0.3493065044544783, 0.049917901555403446, -0.34798101813456656, -0.5051378558082326, 0.17167486498769158, 0.5392602170232761, -0.06863682023767903, 0.7714162025268645, 0.6041928601082343, 0.8356742471671944, -0.6597102371274987, -0.29816687392842656, -0.9135505724111677, -0.5126574643445919, -0.434931214141813, -0.01709391019610479, 0.4855749818219295, 0.8815691614398556, -0.3781483168687406, -0.18945726745083463, -0.08911044324850244, -0.09559615033778196, 0.6185685691196648, 0.32037295514643804, -0.04449663384612053, 0.1998345097128906, 0.9223722503447906, -0.1734814851683703, -0.6518219576548921, 0.27096567717469733, 0.11159977403333604, 0.4959729166418325, 0.7125040071305416, -0.5260765562337709, -

In [61]:
a = he_layer.serialize()

In [62]:
new_he = ts.ckks_vector_from(context_server, a)
new_he += 100
b = new_he.serialize()

In [66]:
resp = ts.ckks_vector_from(context_client, b)
reshaped_model  = []
decrypted_model = resp.decrypt()

for p in model.get_weights():
    reshaped_model.append(np.reshape(decrypted_model[:p.size], p.shape))
    decrypted_model = decrypted_model[p.size:]

In [67]:
model.set_weights(reshaped_model)

In [68]:
model.get_weights()

[array([[100.793816, 100.61136 , 100.65261 ,  99.22408 , 100.45512 ,
          99.87493 , 100.6384  , 100.26041 , 100.40495 ,  99.52592 ],
        [ 99.61784 ,  99.83002 ,  99.25724 , 100.55962 ,  99.650696,
         100.04992 ,  99.652016,  99.494865, 100.17168 , 100.53926 ],
        [ 99.931366, 100.771416, 100.604195, 100.83567 ,  99.34029 ,
          99.701836,  99.08645 ,  99.48734 ,  99.56507 ,  99.9829  ],
        [100.48557 , 100.88157 ,  99.62185 ,  99.81054 ,  99.91089 ,
          99.9044  , 100.61857 , 100.32037 ,  99.955505, 100.19984 ],
        [100.92237 ,  99.826515,  99.348175, 100.270966, 100.1116  ,
         100.49597 , 100.7125  ,  99.47392 ,  99.21962 ,  99.44331 ],
        [ 99.422356,  99.189926, 100.46748 ,  99.224724,  99.86569 ,
          99.5466  , 100.23616 , 100.834625,  99.53998 ,  99.07956 ],
        [ 99.54361 ,  99.99875 ,  99.661705, 100.3322  , 100.81418 ,
         100.839806, 100.67901 ,  99.12153 , 100.77568 ,  99.30874 ],
        [100.48835 ,  99.52

Traceback (most recent call last):
  File "c:\Users\allan\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 10, in <module>
  File "C:\Python310\lib\timeit.py", line 234, in timeit
    return Timer(stmt, setup, timer, globals).timeit(number)
  File "C:\Python310\lib\timeit.py", line 130, in __init__
    raise ValueError("stmt is neither a string nor callable")
ValueError: stmt is neither a string nor callable



In [ ]:
a

Traceback (most recent call last):
  File "c:\Users\allan\.vscode\extensions\ms-python.python-2024.20.0-win32-x64\python_files\python_server.py", line 130, in exec_user_input
    retval = callable_(user_input, user_globals)
  File "<string>", line 1, in <module>
NameError: name 'a' is not defined

